In [1]:
import torch as t
import torch 
import pickle
from torch import nn, Tensor 
import torch, math   
from torch.utils.data import DataLoader, TensorDataset
import os
import glob
from utils import PositionalEncoding

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print(torch.cuda.get_device_name(0)) 

class DREAMLazyTestData(TensorDataset):
    def __init__(self, device=device):
        super().__init__()
        self.path = os.getcwd() + '/pytorch-tensors-test-set/'
        
    def __getitem__(self, item):
        zero_pad = (5 - len(str(item)))
        basename = 'Batch' + ('0'*zero_pad) + str(item)
       
        Xfile = basename + 'X.pt'
        X = t.load(self.path + Xfile).to(device)
        return X
    
    def __len__(self):
         return (len(glob.glob1(self.path, "*.pt")))

NVIDIA GeForce GTX TITAN X


In [2]:
myData = DREAMLazyTestData()
myNet = torch.load(os.getcwd() + '/DREAMnet.pt')

y_pred, y = [], []

def evaluate(myModel, myData, has_labels = False):
    nfiles = len(glob.glob1(myData.path, "*.pt"))
    
    for i in range(nfiles): 
        if i % 10 == 0: print(i)
        
        if has_labels: 
            test_X, _test_y = myData.__getitem__(i)
            y.append(_test_y) 
        else:     
            test_X = myData.__getitem__(i)
        
        with t.no_grad():  
            y_pred.append(myModel(test_X).reshape(-1))
 
    y = t.hstack(y).cpu().numpy() if has_labels == True else [] 

    return t.hstack(y_pred).cpu().numpy(), y

In [3]:
y_pred, _ = evaluate(myNet, myData)
print("Done!")

0
10
20
30
40
50
60
70
Done!


In [4]:
import numpy as np 
import json

test = np.genfromtxt(os.getcwd() + '/test_sequences.txt', dtype=None, encoding=None)
assert test.shape == y_pred.shape

pred_dict = {test[k][0]: float(y_pred[k]) for k in range(len(y_pred))}


In [5]:
with open('submission.json', 'w') as fp:
    json.dump(pred_dict, fp)

In [17]:
import pandas as pd 
df = pd.DataFrame(list(zip( [test[k][0] for k in range(len(y_pred))], y_pred)))
df.to_csv('submission.txt', sep='\t', index=False, header=False)

In [6]:
print(myNet)

Sequential(
  (0): PositionalEncoding()
  (1): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
    (linear1): Linear(in_features=100, out_features=1000, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (linear2): Linear(in_features=1000, out_features=100, bias=True)
    (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.0, inplace=False)
    (dropout2): Dropout(p=0.0, inplace=False)
  )
  (2): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
    )
    (linear1): Linear(in_features=100, out_features=1000, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (linear2): Linear(in_features=1000, out_features=100, bias=True)
    (norm1): LayerNorm